In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
from lion_pytorch import Lion
import json

from vae import VAE
from trainer import Finetuner

In [3]:
# Parameters to set:
n_synthetic_samples = 500
data_name = 'one_hot_pNaNs_agep.csv'
model_name = 'best_model_pNaNs_agep.pth'

latent_dim = 500
hidden_dim = 1500

In [4]:
# Device config 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")


# load data and marginals
data = pd.read_csv(f'workspace/data/{data_name}')
data_tensor = torch.tensor(data.values, dtype=torch.float32)

with open('marginals.json') as f:
    marginals = json.load(f)

# Load model
group_sizes = [2, 5, 5, 11, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]

model = VAE(433, hidden_dim, 6, latent_dim, group_sizes)

params = torch.load(f'workspace/models/{model_name}', map_location=torch.device('cpu') )
model.load_state_dict(params)

# Generate synthetic codes - make trainable and put in optimizer
trainable_latent_codes = torch.randn(n_synthetic_samples, latent_dim).to(device)
trainable_latent_codes.requires_grad = True

optimizer = optim.AdamW([trainable_latent_codes], lr = 1e-3)


Device: cpu


/tmp/ipykernel_142272/757797158.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  params = torch.load(f'workspace/models/{model_name}', map_location=torch.device('cpu') )

In [5]:
# Initialise finetuner
column_names = list(data.columns)

finetuner = Finetuner(column_names, marginals, model, optimizer, device)

AttributeError: 'list' object has no attribute 'update'